In [1]:
import torch
#from segment_anything import SamPredictor, sam_model_registry
from PIL import Image
import numpy as np
import kagglehub


In [2]:
import os
import random
import shutil

# Caminhos para os diretórios
base_dir = "D:\\T.I\\BootcampAvanti_ML\\DeteccaoMelanoma_ML"
train_dir = os.path.join(base_dir, 'train')
new_dataset_dir = os.path.join(base_dir, 'dataset_with_masks')

# Classes a serem processadas
classes = ['Benign', 'Malignant']

# Função para criar o novo dataset com imagens aleatórias
def create_dataset(input_dir, output_dir, sample_size=50):
    for class_name in classes:
        class_dir = os.path.join(input_dir, class_name)
        dest_class_dir = os.path.join(output_dir, class_name)
        
        if not os.path.exists(dest_class_dir):
            os.makedirs(dest_class_dir)

        # Selecionar aleatoriamente uma amostra de imagens
        img_names = os.listdir(class_dir)
        random_img_names = random.sample(img_names, sample_size)
        
        for img_name in random_img_names:
            img_path = os.path.join(class_dir, img_name)
            dest_img_path = os.path.join(dest_class_dir, img_name)
            
            if os.path.isfile(img_path):
                try:
                    # Copiar a imagem para o novo diretório
                    shutil.copy(img_path, dest_img_path)
                    print(f"Imagem {img_name} copiada para {dest_class_dir}")
                except Exception as e:
                    print(f"Erro ao processar a imagem {img_name}: {e}")

# Criar o novo dataset com as imagens
create_dataset(train_dir, new_dataset_dir, sample_size=150)


Imagem 526.jpg copiada para D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign
Imagem 5753.jpg copiada para D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign
Imagem 5532.jpg copiada para D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign
Imagem 2390.jpg copiada para D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign
Imagem 3822.jpg copiada para D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign
Imagem 4668.jpg copiada para D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign
Imagem 1372.jpg copiada para D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign
Imagem 894.jpg copiada para D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign
Imagem 5986.jpg copiada para D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign
Imagem 2406.jpg copiada para D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign
Im

In [3]:
import kagglehub

# Download latest version
path = kagglehub.model_download("metaresearch/segment-anything/pyTorch/vit-l")

print("Path to model files:", path)

Path to model files: C:\Users\User\.cache\kagglehub\models\metaresearch\segment-anything\pyTorch\vit-l\1


In [4]:
import os
import numpy as np
from PIL import Image
from segment_anything import SamPredictor, sam_model_registry

# Caminho para o modelo baixado
model_path = "C:/Users/User/.cache/kagglehub/models/metaresearch/segment-anything/pyTorch/vit-l/1/model.pth"

# Carregar o modelo
sam = sam_model_registry["vit_l"](checkpoint=model_path)

# Configurar o preditor
predictor = SamPredictor(sam)

# Caminho para o diretório base
base_dir = "D:\\T.I\\BootcampAvanti_ML\\DeteccaoMelanoma_ML\\"
new_dataset_dir = os.path.join(base_dir, 'dataset_with_masks')

# Classes a serem processadas
classes = ['Benign', 'Malignant']

# Diretórios para salvar as máscaras
mask_dirs = {
    'Benign': os.path.join(new_dataset_dir, 'Benign_Mask'),
    'Malignant': os.path.join(new_dataset_dir, 'Malignant_Mask')
}

# Função para carregar e preparar a imagem
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return np.array(image)

# Função para gerar máscaras
def generate_masks(image):
    predictor.set_image(image)
    masks, scores, logits = predictor.predict()
    return masks

# Função para salvar as máscaras
def save_masks(masks, output_dir, img_name):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for i, mask in enumerate(masks):
        mask_image = Image.fromarray((mask * 255).astype(np.uint8))
        mask_image.save(os.path.join(output_dir, f"{os.path.splitext(img_name)[0]}_mask_{i}.png"))

# Função para processar imagens de um diretório e salvar máscaras
def process_images(input_dir):
    for class_name in classes:
        class_dir = os.path.join(input_dir, class_name)
        dest_mask_dir = mask_dirs[class_name]
        
        if not os.path.exists(dest_mask_dir):
            os.makedirs(dest_mask_dir)

        # Iterar sobre todas as imagens
        img_names = os.listdir(class_dir)
        
        for img_name in img_names:
            img_path = os.path.join(class_dir, img_name)
            if os.path.isfile(img_path):
                try:
                    # Carregar e preparar a imagem
                    img = load_image(img_path)
                    
                    # Gerar máscaras
                    masks = generate_masks(img)
                    
                    # Salvar as máscaras
                    save_masks(masks, dest_mask_dir, img_name)
                    
                    print(f"Máscaras salvas para {img_name} em {dest_mask_dir}")
                except Exception as e:
                    print(f"Erro ao processar a imagem {img_name}: {e}")

# Processar todas as imagens no novo dataset
process_images(new_dataset_dir)


C:\Users\User\AppData\Roaming\Python\Python311\site-packages\segment_anything\build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


Máscaras salvas para 1088.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign_Mask
Máscaras salvas para 1202.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign_Mask
Máscaras salvas para 1338.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign_Mask
Máscaras salvas para 1342.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign_Mask
Máscaras salvas para 1370.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign_Mask
Máscaras salvas para 1372.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign_Mask
Máscaras salvas para 138.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign_Mask
Máscaras salvas para 1384.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign_Mask
Máscaras salvas para 1417.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\dataset_with_masks\Benign_Mask
Máscaras salvas para

KeyboardInterrupt: 

In [ ]:
# def predict_and_save_masks(model, input_dir, output_dir):
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
    
#     for img_name in os.listdir(input_dir):
#         img_path = os.path.join(input_dir, img_name)
#         if os.path.isfile(img_path):
#             try:
#                 # Carregar e pré-processar a imagem
#                 img = cv2.imread(img_path)
#                 preprocessed = preprocess_image(img)
                
#                 # Fazer previsão
#                 mask = model.predict(preprocessed)[0]
                
#                 # Aplicar o threshold
#                 mask = (mask > 0.5).astype(np.uint8) * 255
                
#                 # Salvar a máscara
#                 dest_path = os.path.join(output_dir, img_name)
#                 cv2.imwrite(dest_path, mask)
#                 print(f"Máscara salva em: {dest_path}")
#             except Exception as e:
#                 print(f"Erro ao processar a imagem {img_name}: {e}")
